<a href="https://colab.research.google.com/github/MerkulovDaniil/optim/blob/master/assets/Notebooks/LLM_fine_tuning_on_songs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎤 Давайте дообучим ruGPT3 на текстах песен

В этом практикуме мы увидим, как можно дообучать современные языковые модели на текстах песен.

<img src="https://raw.githubusercontent.com/oseledets/dl2023/main/seminars/seminar-10/neuro_kish.jpg" alt= “” width="128px">

## 👨‍🍳 Пререквизиты

In [ ]:
# Загрузим датасеты
!wget -N https://github.com/MerkulovDaniil/mipt21/raw/main/assets/lyrics.zip
!unzip -q -o lyrics.zip -d ./

!pip install -q sentencepiece
!pip install -q transformers  datasets
!pip install -q accelerate
!pip install -q deepspeed mpi4py
!pip install -q pynvml
!pip install -q wandb

import torch
import pandas as pd
import time

class Profiler():

    def __init__(self,) -> None:
        pass

    def gpu_mem(self):
        mem = torch.cuda.mem_get_info()
        mb = list(map(lambda x:x/pow(2,20),mem))
        total = mb[1]
        used = mb[1]-mb[0]
        return used,total

    def gpu_mem_info(self,title = ''):
        used,total = self.gpu_mem()
        print(f'🤖 {title} gpu mem : {used:.1f}/{total:.1f} mb')

    def one_step_report(self,batch, model, optimizer, do_backward = True,device = torch.device('cpu'),print_loss = False,deepspeed = False):
    
        report_df = pd.DataFrame(columns=['used_mem','delta_mem','delta_time'])

        delta_time =[0]
        used_mem = [self.gpu_mem()[0]]

        self.gpu_mem_info('begin')

        model.train()
        
        ids = batch['input_ids'].to(device,dtype=torch.long)
        labels = batch['labels'].to(device,dtype=torch.long)
        
        torch.cuda.synchronize()
        start_time = time.time()

        outputs = model(input_ids = ids,labels = labels)
        loss = outputs[0]

        torch.cuda.synchronize()
        forward_time = time.time()
        delta_time.append(-start_time + forward_time)

        used_mem.append(self.gpu_mem()[0])
        self.gpu_mem_info(f'{delta_time[-1]:.3f}s forward')
        if do_backward:
            optimizer.zero_grad()
            if deepspeed:
                model.backward(loss)
            else:
                loss.backward()

            torch.cuda.synchronize()
            backward_time = time.time()
            delta_time.append(-forward_time + backward_time)
            used_mem.append( self.gpu_mem()[0])
            self.gpu_mem_info(f'{delta_time[-1]:.3f}s backward')

            if deepspeed:
                model.step()
            else:
                optimizer.step()

            torch.cuda.synchronize()
            optimizer_step_time = time.time()
            delta_time.append(-backward_time + optimizer_step_time)
            used_mem.append( self.gpu_mem()[0])
            self.gpu_mem_info(f'{delta_time[-1]:.3f}s optimizer_step')
        
        if (print_loss):
            print('loss',loss)

        torch.cuda.empty_cache() 
        used_mem.append( self.gpu_mem()[0])
        torch.cuda.synchronize()
        end_time = time.time()
        delta_time.append(end_time - optimizer_step_time)
        # 
        report_df.loc[:,'used_mem'] = pd.Series(used_mem)
        report_df.loc[:,'delta_time'] = pd.Series(delta_time)
        indexes = ['begin','forward','backward','optim_step','end']
        report_df.index = indexes

        report_df['delta_mem'] =  report_df['used_mem']- report_df.loc['begin','used_mem']

        report_df.loc['total'] = [self.gpu_mem()[1],0,end_time-start_time]
        report_df['delta_time'] = report_df['delta_time'].map(lambda t : round(t,3))
        
        return report_df

prof = Profiler()

prof.gpu_mem()
# (5804.0, 15109.75)

prof.gpu_mem_info() 
# gpu mem : 5840.0/15109.8 mb


# report = prof.one_step_report(batch, model,optim,device = DEVICE)
# # begin gpu mem : 5804.0/15109.8 mb
# # 0.050s forward gpu mem : 13006.0/15109.8 mb
# # 1.232s backward gpu mem : 14576.0/15109.8 mb
# # 0.025s optimizer_step gpu mem : 14576.0/15109.8 mb

# report

# Simplest memory profiling
def gpu_mem():
    mem = torch.cuda.mem_get_info()
    mb = list(map(lambda x:x/pow(2,20),mem))
    total = mb[1]
    used = mb[1]-mb[0]
    return used,total

def gpu_mem_info(title = ''):
    used,total = gpu_mem()
    print(f'🤖 {title} gpu mem : {used:.1f}/{total:.1f} mb')

## 📦 Выберем датасет и модель

In [9]:
MODEL_NAME = 'ai-forever/rugpt3small_based_on_gpt2' #@param ['ai-forever/rugpt3small_based_on_gpt2', 'ai-forever/rugpt3medium_based_on_gpt2','ai-forever/rugpt3large_based_on_gpt2', 'gpt2-large']
DATASET_PATH = './\u041B\u0435\u043D\u0438\u043D\u0433\u0440\u0430\u0434_lyrics.txt'  #@param ['./manowar_lyrics.txt','./kish_lyrics.txt','./korzh_lyrics.txt', './oxxxy_lyrics.txt', './pushkin.txt', './Алла Пугачёва_lyrics.txt', './Ария_lyrics.txt', './Виа ГРА_lyrics.txt', './Владимир Высоцкий_lyrics.txt', './Дима Билан_lyrics.txt', './Заточка_lyrics.txt', './Кино_lyrics.txt', './Ленинград_lyrics.txt', './Михаил Круг_lyrics.txt', './Скриптонит_lyrics.txt']

with open(DATASET_PATH) as text_file:
    text = text_file.read().splitlines()
    print(text)

['', 'Вояж', 'Вояж', 'Вояж', 'Вояж', '', 'Все твои подруги — суки, часто ездят за границу', 'Чтобы фоточки в Фэйсбуке залепить, как говорится', 'В Инстаграме свои рожи похудевшие хуячат', 'Мы с тобой поедем тоже, по путевке, по горящей', '', 'На фоне Эйфелевой башни', 'С Айфона селфи заебашим', 'А нахуя ж ещё нам наш вояж?', 'На фоне Эйфелевой башни', 'С Айфона селфи заебашим', 'А нахуя ж ещё нам наш вояж?', '', 'Заебашим мы не хуже, чем твои эти паскуды', 'Устриц будем жрать на ужин, в ресторане, из посуды', 'Полетим на самолете, купим в "дьютике" напитки', 'Мол, не только вы, блядь, пьёте джины с тоником по скидке', 'На фоне Эйфелевой башни', 'С Айфона селфи заебашим', 'А нахуя ж ещё нам наш вояж?', '', 'На фоне Эйфелевой башни', 'С Айфона селфи заебашим', 'А нахуя ж ещё нам наш вояж?', '', 'Бля, Серёг, ну, ты, реально, ахуительный мужик!', 'То баб водишь на Ван Гога, то возишь в Геленджик', 'А теперь ещё с Парижем: чую, вскоре хлебну горя', 'Ведь могу свозить свою я, разве что ли, в

## 💃 Model loading

In [90]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
import transformers
from sklearn.model_selection import train_test_split
import time
import pandas as pd
import random
import deepspeed

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"🤖 Working on {DEVICE}")

model_name_or_path = MODEL_NAME
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).to(DEVICE)

🤖 Working on cuda


In [91]:
print(f"🤖The total number of parameters in the model is {model.num_parameters()}")
gpu_mem_info()

🤖The total number of parameters in the model is 125231616
🤖  gpu mem : 14821.0/15101.8 mb


## 🎁 Взгляд на данные

In [92]:
if 'pushkin' in DATASET_PATH or 'mayakovskiy' in DATASET_PATH:
    tokenizer.add_tokens('</s>')
    tokenizer.add_special_tokens({
        'eos_token': '</s>',
        'pad_token': '<pad>'
    })
if 'lyrics' in DATASET_PATH:
    tokenizer.add_tokens('[EOS]')
    tokenizer.add_special_tokens({
        'eos_token': '[EOS]',
        'pad_token': '<pad>'
    })

model.resize_token_embeddings(len(tokenizer))

# Dataset
train_dataset = TextDataset(tokenizer=tokenizer,file_path=DATASET_PATH,block_size=512)
train_dataset, eval_dataset = train_test_split(train_dataset,test_size = 0.1,random_state = 42)
  
# Creating a data_collator (slices the text into optimal length pieces)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [93]:
import numpy as np
print(f"🤖 Размер словаря (количество различных токенов): {tokenizer.vocab_size}")
print(f"🤖 Случайные примеры токенов: {[tokenizer.decode(id) for id in np.random.choice(range(tokenizer.vocab_size), 4)]}")

🤖 Размер словаря (количество различных токенов): 50257
🤖 Случайные примеры токенов: ['duct', ' M', ' большинству', ' подходящие']


In [94]:
print(f"🤖 Как выглядит первый батч из обучающей выборки размера {len(train_dataset[0])}")
print(train_dataset[0])

🤖 Как выглядит первый батч из обучающей выборки размера 512
tensor([  203,  1916, 19405,   912,  1209,   289,   352,   296,   400,     5,
          203,  3096,  9040, 11251,   203,  6014, 11850,   332,   289,  8168,
         6399,   203,   203,   682,   489,   847,   773,   670,  1003,   203,
        13762,  4174,  4287,  6631,  2172,   203,   203,  2176,  2959,   271,
          694,   670,  3808,   203,  6115,   501, 11015,   454,   322,   352,
          818,   203,   789, 16880,  3190,   309, 29197,   452,   203,  3693,
          598, 14602, 19749,   323,   575,     5,   203,  2176,  2959,   271,
          694,   670,  3808,   203,  6115,   501, 11015,   454,   322,   352,
          818,   203, 26635,   296,   454,  1320,  4155,   400,   203, 39169,
           16, 37507,    16, 37507,   203, 39169,    16, 12876,     5, 50257,
          789,  1486,  2403,   289, 27133,  6779,  1203,   203,   677, 39223,
          349,  1805,    16,  1851,  1588,  1003,   203,   630,   755, 38007,
    

In [95]:
# Testing
tokenizer.decode(train_dataset[0])

'\nХватит есть всех и ебсти!\nНе получится уха\nИз козла и петуха\n\nНравится им или нет\nЕвропейцам дам совет\n\nНатурал ты или би\nОбезьянку не еби\nТормози на кураже\nЗаразишься оспой же!\nНатурал ты или би\nОбезьянку не еби\nРыбку тоже отпусти\nГосподи, господи, господи\nГосподи, прости! [EOS] Творческий и креативный люд\nВ панике за жизнь, какой уж нет\nОторвавшись от винца и блюд\nПлачутся о судьбах в интернет\n\nО курортах, виллах и счетах\nТех, что превратили разом в пыль\nНе война страшит их, а счета\nИ накопленных ничтожность миль\n\nНу, ты куда ж, кумир\nОй, миль, пардон\nС криками "За мир!"\nСъебал он за кордон\n\nФинита ля комедь-комедь, финита\nЗдесь можно охуеть, прощай, элита\n\nПолюбасу время кончилось колбас\nФорумов, фуршетов, биеннале\nК нам самим пришёл полный Донбасс\nИ пиздец всем гарантирован в финале\n\nГуманисты, в прочем, ведь, и я же\nКто сейчас вспомнит смерть Саддам Хусейна\nБарышня при полном макияже\nФотку скорби постит у бассейна\nYou might also likeНу,

## 👨‍🏫 Обучение

In [96]:
batch_size = 4
n_epochs = 15
training_args = TrainingArguments(
    output_dir="./finetuned", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=n_epochs, # number of training epochs
    warmup_steps=10,# number of warmup steps for learning rate scheduler
    gradient_accumulation_steps=4, # to make "virtual" batch size larger
    evaluation_strategy  = 'epoch',
    logging_strategy  = 'epoch',
    save_strategy  = 'epoch',
    auto_find_batch_size = True,
    load_best_model_at_end = True,
    report_to="none",
    )


In [99]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset = eval_dataset,
    # optimizers = (torch.optim.NAdam(model.parameters(),lr=1e-5),None) # Optimizer and lr scheduler
)

In [100]:
trainer_log = trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
0,3.231400,2.233701
1,2.368400,2.201550
2,2.241400,2.179214
4,2.091700,2.165327
4,2.731600,2.169018
5,1.996100,2.169128
6,1.942300,2.168176
8,1.905200,2.168822
8,2.527900,2.168822
9,1.915500,2.168822


## 🗣️ Генерация

In [114]:
import torch

text = "КОГДА ПЕРЕЕХАЛ - НЕ ПОМНЮ! \nНАВЕРНОЕ, БЫЛ Я БУХОЙ! \nМОЙ АДРЕС СЕГОДНЯ ТАКОЙ\n"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
attention_mask = torch.ones_like(input_ids)  # Set all values to 1 initially
pad_token_id = 50256  # EOS token ID

# Ensure that the attention mask and pad token ID are set
if pad_token_id is not None:
    attention_mask[input_ids == pad_token_id] = 0  # Set attention mask to 0 for padding tokens

# Generate output using the model
model.eval()
with torch.no_grad():
    out = model.generate(input_ids,
                         do_sample=True,
                         num_beams=4,
                         temperature=2.5,
                         top_p=0.9,
                         max_length=200,
                         attention_mask=attention_mask
                         )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



КОГДА ПЕРЕЕХАЛ - НЕ ПОМНЮ! 
НАВЕРНОЕ, БЫЛ Я БУХОЙ! 
МОЙ АДРЕС СЕГОДНЯ ТАКОЙ
ПЯТОЙ ГОРОДСКОЙ  ПОДРОБНЕЙШЕЙ ЧАЙНИ
(На фото - мой дом и моя собака!)
ПЕСНИ СО СТРОКОМ (Твоей, мой, моей любимой собачке)
А ПОЭТ-ЗОЖ (ЕЕ, моей любимой кошке)
Я люблю тебя, мой прекрасный
За что люблю — за что не знаю!
И если хочешь меня понять
Если не веришь в то, что это может быть —
Поделись, может, в себе ты хоть иногда
Пой, сука, со мной своими паяльниками
В них есть, что терять!

Я люблю твою собаку!
Мой милый и милый, мой лучший
Прикольно


## Замеры памяти

In [72]:
b_size = 2
random.shuffle(train_dataset)
batch = data_collator(train_dataset[:b_size])
optim = torch.optim.Adam(model.parameters(),lr=1e-5)

prof = Profiler()
prof.one_step_report(batch, model,optim,device = DEVICE)

🤖 begin gpu mem : 13071.0/15101.8 mb
🤖 0.144s forward gpu mem : 13071.0/15101.8 mb
🤖 0.229s backward gpu mem : 13071.0/15101.8 mb
🤖 0.055s optimizer_step gpu mem : 13071.0/15101.8 mb


<ipython-input-10-4a985b297fe6>:88: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  report_df.loc[:,'delta_time'] = pd.Series(delta_time)


,used_mem,delta_mem,delta_time
begin,13071.0000,0.0,0.000
forward,13071.0000,0.0,0.144
backward,13071.0000,0.0,0.229
optim_step,13071.0000,0.0,0.055
end,4271.0000,-8800.0,0.134
total,15101.8125,0.0,0.562


## Useful links

1. [Tokenizers tutorial](https://huggingface.co/docs/transformers/tokenizer_summary) - brief analysis of all types of tokenizers from Huggingface with examples.
1. [How to generate text](https://huggingface.co/blog/how-to-generate) - overview of how to sample text using language models (bimsurch, etc.).
1. [Attention is All You Need](https://arxiv.org/pdf/1706.03762.pdf) - original article about the first Transformer.
1. [GPT-1](https://openai.com/blog/language-unsupervised/) - an article on OpenAI blog about GPT-1.
1. [GPT-2](https://openai.com/blog/better-language-models/) - OpenAI blog article about GPT-2.
1. [GPT-3](https://openai.com/blog/gpt-3-apps/) - OpenAI blog article about GPT-3.
1. [WebGPT](https://openai.com/blog/improving-factual-accuracy/) - OpenAI blog article about GPT-3, trained to google.
1. [Codex](https://openai.com/blog/openai-codex/) - OpenAI blog article about GPT-3 trained to write code.